In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
Y = np.array([[3],[1],[8],[3],[5]])
Y

array([[3],
       [1],
       [8],
       [3],
       [5]])

In [3]:
X = np.array([[1,3],[1,1],[1,5],[1,2],[1,4]])
X

array([[1, 3],
       [1, 1],
       [1, 5],
       [1, 2],
       [1, 4]])

In [4]:
XT_X = np.matmul(np.matrix_transpose(X), X)
XT_X

array([[ 5, 15],
       [15, 55]])

In [5]:
XT_X_inv = np.linalg.inv(XT_X)
XT_X_inv

array([[ 1.1, -0.3],
       [-0.3,  0.1]])

In [6]:
XT_Y = np.matmul(np.matrix_transpose(X), Y)
XT_Y

array([[20],
       [76]])

In [7]:
betas = np.matmul(XT_X_inv, XT_Y)
betas

array([[-0.8],
       [ 1.6]])

In [8]:
# calculo de Y pronosticadas
y_pred = np.matmul(X, betas)
y_pred

array([[4. ],
       [0.8],
       [7.2],
       [2.4],
       [5.6]])

In [9]:
residuales = Y - y_pred
residuales

array([[-1. ],
       [ 0.2],
       [ 0.8],
       [ 0.6],
       [-0.6]])

In [10]:
# Suma de residuos al cuadrado 
# calculo de la bondad de ajuste con funciones de R2
RSS = np.matmul(np.matrix_transpose(residuales), residuales)
RSS

array([[2.4]])

In [11]:
# calculo de la suma total de cuadrados
TSS = np.matmul(np.matrix_transpose(Y - np.mean(Y)), Y - np.mean(Y))
TSS

array([[28.]])

In [12]:
# calculo de la suma total de cuadrados de otra manera
TSS2 = np.matmul(np.matrix_transpose(Y), Y) - len(Y) * (Y.mean() ** 2)
TSS2

array([[28.]])

In [13]:
# calculo del coeficiente de determinacion
R_cuad = float(1 - (RSS / TSS))
R_cuad

0.9142857142857144

In [14]:
# calculo de la varianza de los errores
S_cuad = RSS / (len(Y) - X.shape[1])
S_cuad

array([[0.8]])

In [15]:
# desviacion estandar de los errores
S = np.sqrt(S_cuad)
S

array([[0.89442719]])

In [16]:
# obtencion del valor critico de la t de student
import scipy.stats 

# grados de libertad : n - (k+1)
grados_libertad = len(Y) - X.shape[1]
confianza = 0.95
q = 1 - (1 - confianza) / 2
t_critico = abs(scipy.stats.t.ppf(q, grados_libertad))
t_critico

np.float64(3.182446305284263)

In [17]:
# vector de valores particulares de X
f = np.array([[1], [7]])
f

array([[1],
       [7]])

In [18]:
margen_error = t_critico * (S *(float(np.matmul(np.matmul(np.matrix_transpose(f),XT_X_inv), f) ** 0.5)))
margen_error

array([[3.81893557]])

In [19]:
pron_puntual = float(np.matmul(np.matrix_transpose(f), betas))
pron_puntual

10.400000000000006

In [20]:
# limites de intervalo de confianza
Lim_inf = pron_puntual - margen_error
Lim_sup = pron_puntual + margen_error
print(f"El intervalo de confianza es: {Lim_inf}, {Lim_sup}")

El intervalo de confianza es: [[6.58106443]], [[14.21893557]]


VALIDACION DE SUPUESTOS DE LA REGRESIÓN

In [21]:
import scipy
residuales

array([[-1. ],
       [ 0.2],
       [ 0.8],
       [ 0.6],
       [-0.6]])

In [22]:
# calculo de la simetría de los residuos
skev = float(scipy.stats.skew(residuales, bias = True))
skev

-0.2886751345948135

In [23]:
# calculo de la curtosis de los residuales
kurtosis = float(scipy.stats.kurtosis(residuales, fisher = False))
kurtosis

1.4499999999999993

In [24]:
Jarque_Bera = (len(Y) / 6) * (skev ** 2 + ((kurtosis - 3) ** 2) / 4)
Jarque_Bera

0.5699652777777785

In [25]:
import scipy.stats


Nivel_confianza = 0.95
scipy.stats.chi2.ppf(Nivel_confianza, df = 2)

np.float64(5.991464547107979)

Conclusión: dado que JB no es mayor al nivel critico, no podemos rechazar la hipotesis de existencia de normalidad en los residuales

### Supuesto 2: Inexistencia de autocorrelación entre residuales

In [26]:
from statsmodels.formula.api import ols

In [27]:
y_df = pd.DataFrame(Y)
x_df = pd.DataFrame(X[:,1:2])
x_df

,0
0,3
1,1
2,5
3,2
4,4


In [28]:
df = pd.concat((y_df,x_df.reindex(y_df.index)), axis=1)
df.columns = ['y', 'x1']
df

,y,x1
0,3,3
1,1,1
2,8,5
3,3,2
4,5,4


In [29]:
# ajuste de regresion lineal multiple
model = ols('y ~ x1', data=df).fit()

from statsmodels.stats.stattools import durbin_watson

# prueba de durbin watson
durbin_watson(model.resid)

np.float64(1.3666666666666656)

Conclusion: Dado que DW no es aprox. igual a 2, podemos pensar que existe autocorrelacion entre los residuales

## Supuesto 3: Homocedasticidad ( igual de varianzas)

In [31]:
residcuad = residuales ** 2
residcuad = pd.DataFrame(residcuad, columns=['residcuad'])
residcuad

,residcuad
0,1.00
1,0.04
2,0.64
3,0.36
4,0.36


In [40]:
X1 = df.iloc[:,1]
X1_df = pd.DataFrame(X1.values, columns=['X1'])
X1cuad = X1 ** 2
X1cuad_df = pd.DataFrame(X1cuad.values, columns=['X1cuad'])

In [41]:
df_Aux = pd.concat([residcuad, X1_df, X1cuad_df], axis=1)
df_Aux.columns = ['residual', 'x1', 'X1cuad']
df_Aux

,residual,x1,X1cuad
0,1.00,3,9
1,0.04,1,1
2,0.64,5,25
3,0.36,2,4
4,0.36,4,16


In [42]:
# Ajuste de regresion lineal multiple con cuadrado de X1
modelAux= ols('residuales ~ X1 + X1cuad', data=df_Aux).fit()
RSgAux = modelAux.rsquared
RSgAux

np.float64(0.47619047619047594)

In [43]:
Estadistico = len(Y) * RSgAux
Estadistico

np.float64(2.3809523809523796)

In [44]:
Nivel_confianza = 0.95
scipy.stats.chi2.ppf(Nivel_confianza, df = 2)

np.float64(5.991464547107979)

Conclusion: Al no superar el valor critico nuestro estadistico de prueba, no hay evidencia de heterocedasticidad ( Desigualdad de varianzas de los residuales)

In [46]:
# Alternativa para la de prueba de White

from statsmodels.stats.diagnostic import het_white
white_test = het_white(model.resid, model.model.exog)
print(f"White test statistic: {white_test[0]}, p-value: {white_test[1]}")

White test statistic: 2.66313932980599, p-value: 0.26406244627058784


Conclusion: a un nivel del Alfa=5%, como tenemos un valor p superior a Alfa, no podemos rechazar la hipotesis de Homocedasticidad ( lo cual implica que no existe evidencia de Heterocedasticidad)

# Supuesto 4: Inexistencia de Multicolinealidad

En este caso no aplica realizarla ya que solo tenemos una variable regresora (X), en modelos con mas variables independientes si habria que realizarla